In [1]:
import pandas as pd
from os import listdir
from parsedatetime import parsedatetime
from datetime import datetime
from statistics import mode
# Working directory and date parser
wd = "Z:/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Erin Cobry/Prospective HCL and sleep study"
cal = parsedatetime.Calendar()

In [6]:
# Get file paths
files = listdir(wd + "./Data raw/CGM")
# Iterate through, clean, and calculate variables
results = {"id":[],"total_sensor_readings":[],"percent_cgm_wear":[],
           "percent_time_under_54":[],"time_under_54":[],"num_values_under_54":[]}
for f in files:
    results["id"].append(f)
    # Read in
    path = wd + "/Data raw/CGM/" + f
    df = pd.read_csv(path)
    # Find columns
    time_col = [c for c in df.columns if "times" in c.lower()]
    gluc_col = [c for c in df.columns if "glucose value" in c.lower()]
    # Remove unnecessary columns
    clean = pd.concat([df[time_col],df[gluc_col]],axis = 1)
    clean.columns = ["time","glucose"]
    # Remove missing
    clean = clean[clean['time'].notnull()]
    # Format time
    clean["time"] = clean["time"].str.replace('T',' ')
    clean["time"] = [cal.parse(t)[0] for t in clean["time"]]
    clean["time"] = [datetime(*t[:6]) for t in clean["time"]]
    # Find sampling interval
    s = str(mode(clean["time"].diff()).total_seconds() / 60)
    # Re-index and expand to every 5 minutes
    clean.set_index("time",inplace = True)
    clean = clean[~clean.index.duplicated(keep='first')]
    clean = clean.resample(s + "T").ffill()
    clean.reset_index(inplace = True)
    # Write
    clean.to_csv(wd + "/Data clean/CGM/" + f,index = False)
    # Variables
    results["total_sensor_readings"].append(clean["glucose"].notna().sum())
    results["percent_cgm_wear"].append(round(clean["glucose"].notna().sum() / clean.shape[0] * 100,2))

In [17]:
results["id"].append(f)
# Read in
path = wd + "/Data raw/CGM/" + f
df = pd.read_csv(path)
# Find columns
time_col = [c for c in df.columns if "times" in c.lower()]
gluc_col = [c for c in df.columns if "glucose value" in c.lower()]
# Remove unnecessary columns
clean = pd.concat([df[time_col],df[gluc_col]],axis = 1)
clean.columns = ["time","glucose"]
# Remove missing
clean = clean[clean['time'].notnull()]
# Format time
clean["time"] = clean["time"].str.replace('T',' ')
clean["time"] = [cal.parse(t)[0] for t in clean["time"]]
clean["time"] = [datetime(*t[:6]) for t in clean["time"]]
# Find sampling interval
s = str(mode(clean["time"].diff()).total_seconds() / 60)
# Re-index and expand to every 5 minutes
clean.set_index("time",inplace = True)
clean = clean[~clean.index.duplicated(keep='first')]
clean = clean.resample(s + "T").ffill()
clean.reset_index(inplace = True)

In [20]:
round(clean["glucose"].notna().sum() / clean.shape[0] * 100,2)

99.75

In [14]:
help(clean["glucose"].isna)

Help on method isna in module pandas.core.series:

isna() -> 'Series' method of pandas.core.series.Series instance
    Detect missing values.
    
    Return a boolean same-sized object indicating if the values are NA.
    NA values, such as None or :attr:`numpy.NaN`, gets mapped to True
    values.
    Everything else gets mapped to False values. Characters such as empty
    strings ``''`` or :attr:`numpy.inf` are not considered NA values
    (unless you set ``pandas.options.mode.use_inf_as_na = True``).
    
    Returns
    -------
    Series
        Mask of bool values for each element in Series that
        indicates whether an element is an NA value.
    
    See Also
    --------
    Series.isnull : Alias of isna.
    Series.notna : Boolean inverse of isna.
    Series.dropna : Omit axes labels with missing values.
    isna : Top-level isna.
    
    Examples
    --------
    Show which entries in a DataFrame are NA.
    
    >>> df = pd.DataFrame(dict(age=[5, 6, np.NaN],
    ... 